# Modeling on Roseman Stored Data

In this notebook, we will build predictive models based on the cleaned and processed Roseman Stored dataset.

## Objectives
- Select appropriate features for modeling.
- Split the data into training and testing sets.
- Train various machine learning models.
- Evaluate the models' performance.
- Draw insights and recommendations based on the model results.

*Note:* This notebook follows the data cleaning and visualization steps done previously.

### Importing essential libraries for regression modeling

In [28]:
!pip install xgboost

In [67]:
# For data manipulation
import numpy as np
import pandas as pd

# For data preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.decomposition import PCA

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif
from xgboost import XGBRegressor

# Model evaluation metrics for regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Optional: for saving/loading models
import joblib

# Optional: warnings control
import warnings
warnings.filterwarnings('ignore')

### Helper Function

In [68]:
def data_overview(df, name="DataFrame"):
    """
    Show basic structure of the DataFrame:
    - Shape
    - Column names
    - Data types and non-null values
    """
    print(f"\n--- Overview of {name} ---")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\n--- Info ---")
    print(df.info())


def data_statistics(df, name="DataFrame"):
    """
    Show statistical summary of the DataFrame:
    - Descriptive statistics for all columns
    - Number of unique values per column
    """
    print(f"\n--- Descriptive Statistics of {name} ---")
    print(df.describe(include='all').transpose())
    
    print(f"\n--- Unique Values per Column in {name} ---")
    print(df.nunique().sort_values())


def missing_values_report(df, name="DataFrame"):
    """
    Display a formatted text report of missing values in the DataFrame.
    The output looks like a table, but it's printed as plain text.
    """
    print(f"\n--- Missing Values in {name} ---")
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df)) * 100

    # Keep only columns with missing values
    mask = missing_count > 0
    if mask.sum() == 0:
        print("No missing values detected.")
        return

    # Print table header
    print(f"{'':<18}{'Missing Count':>15}  {'Missing %':>10}")

    # Print each row aligned
    for col in df.columns[mask]:
        count = missing_count[col]
        percent = missing_percent[col]
        print(f"{col:<18}{count:>15}  {percent:>10.6f}")


def show_value_counts(df, columns):
    """
    Display value counts for a list of columns in a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame
    columns (list): A list of column names for which to show value counts
    """
    for col in columns:
        print(f"--- Value Counts for column: '{col}' ---")
        print(df[col].value_counts())
        print("-" * 50)


def train_evaluate_model(model, x_train, y_train, x_test, y_test):
    """
    Train the given model on training data, predict on both training and test data,
    then calculate and print MAE, RMSE, and R2 metrics for both datasets.
    
    Parameters:
    - model: sklearn-compatible regression model instance
    - x_train: training features
    - y_train: training targets
    - x_test: test features
    - y_test: test targets
    """
    model.fit(x_train, y_train)
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    mae_train = mean_absolute_error(y_train, y_train_pred)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    mae_test = mean_absolute_error(y_test, y_test_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    print(f"{model.__class__.__name__} Performance Metrics\n")
    print("Training Set:")
    print(f"  Mean Absolute Error (MAE): {mae_train:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse_train:.4f}")
    print(f"  R-squared Score (R2): {r2_train:.4f}\n")
    print("Test Set:")
    print(f"  Mean Absolute Error (MAE): {mae_test:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse_test:.4f}")
    print(f"  R-squared Score (R2): {r2_test:.4f}")

### Load Cleaned Datasets for Further Processing

In [69]:
# Load cleaned training data
clean_data = pd.read_csv(r"D:\Work\CODING\PYTHON\DEPI-project\data\clean_data.csv")

# Load raw test data
test_data = pd.read_csv(r"D:\Work\CODING\PYTHON\DEPI-project\data\test_data.csv")

# Merge both datasets to apply unified preprocessing
df = pd.concat([clean_data, test_data], axis=0).reset_index(drop=True)

### Data Preprocessing: Dropping Irrelevant Columns and Encoding Categorical Variables

In [70]:
# Use data overview function to display basic info about the cleaned data
data_overview(df, name="Full Data")


--- Overview of Full Data ---
Shape: (1058297, 22)
Columns: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'compdistance', 'compmonth', 'compyear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Year', 'Month', 'Day', 'Day_name']

--- Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058297 entries, 0 to 1058296
Data columns (total 22 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Store            1058297 non-null  int64  
 1   DayOfWeek        1058297 non-null  int64  
 2   Date             1058297 non-null  object 
 3   Sales            1017209 non-null  float64
 4   Customers        1058297 non-null  float64
 5   Open             1058297 non-null  int64  
 6   Promo            1058297 non-null  int64  
 7   StateHoliday     1058297 non-null  object 
 8   SchoolHoliday    1058297 non-null  int64  
 9   StoreTy

In [71]:
# Use data statistics function to show descriptive stats and unique values
data_statistics(df, name="Full Data")


--- Descriptive Statistics of Full Data ---
                     count unique         top    freq         mean  \
Store            1058297.0    NaN         NaN     NaN   558.331493   
DayOfWeek        1058297.0    NaN         NaN     NaN     3.997596   
Date               1058297    990  2015-07-31    1115          NaN   
Sales            1017209.0    NaN         NaN     NaN  5773.818972   
Customers        1058297.0    NaN         NaN     NaN    -1.435253   
Open             1058297.0    NaN         NaN     NaN     0.831048   
Promo            1058297.0    NaN         NaN     NaN      0.38207   
StateHoliday       1058297      5           0  625531          NaN   
SchoolHoliday    1058297.0    NaN         NaN     NaN     0.188929   
StoreType          1058297      4           a  573755          NaN   
Assortment         1058297      3           a  557749          NaN   
compdistance     1058297.0    NaN         NaN     NaN    -0.048868   
compmonth        1058297.0    NaN         NaN

### Preprocessing: Dropping & Encoding

In [81]:
# Generate a missing values report for the cleaned dataset
missing_values_report(df, name="Full Data")


--- Missing Values in Full Data ---
                    Missing Count   Missing %
Sales                       41088    3.882464


In [82]:
# Drop 'Date', 'DayOfWeek' and is_test columns because their information is already captured by 'Year', 'Month', 'Day', and other features.
# Keeping these columns would be redundant.
columns_to_drop = ['Date', 'DayOfWeek', 'is_test']

df.drop(columns=columns_to_drop, axis=1, inplace=True)

KeyError: "['is_test'] not found in axis"

In [ ]:
# Extract columns with categorical data type
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Generate value counts report for categorical columns
show_value_counts(df, categorical_columns)

In [ ]:
# Map StateHoliday categorical values to numerical codes
stateholiday_map = {'0': 0, 'a': 1, 'b': 2, 'c': 3}
df['StateHoliday'] = df['StateHoliday'].map(stateholiday_map)

# Map Assortment categorical values to numerical codes
assortment_map = {'a': 0, 'b': 1, 'c': 2}
df['Assortment'] = df['Assortment'].map(assortment_map)

In [ ]:
# Apply one-hot encoding to 'Day_name', 'StoreType' and 'PromoInterval' columns
# - dtype='int64' ensures resulting dummy variables are integers, which saves memory and is often preferred for ML models
# - drop_first=True avoids dummy variable trap (multicollinearity) by dropping the first category from each encoded column
df = pd.get_dummies(df, columns=['StoreType', 'PromoInterval', 'Day_name'], dtype='int64', drop_first=True)

In [ ]:
# Calculate correlation between each feature and 'Sales'
sales_correlations = df.corr(numeric_only=True)['Sales'].sort_values(ascending=False)

# Display correlations
print("Correlation between each feature and 'Sales':\n")
print(sales_correlations)

In [ ]:
# Drop 'Customers' column because it is missing in test data 
# and is strongly correlated with the target variable 'Sales',
# which makes it redundant and potentially misleading for model training.
df.drop(columns=['Customers'], inplace=True)

In [ ]:
# Split the dataset into train and test sets based on missing values in 'Sales'
# Rows with non-missing 'Sales' will be used for training
# Rows with missing 'Sales' will be used for testing (unlabeled data)
train_data = df[df['Sales'].notnull()].copy()
test_data = df[df['Sales'].isnull()].copy()

# Optional: reset index for both sets
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
# Generate a missing values report for the train and test dataset
missing_values_report(train_data, name="Train Dataset")
missing_values_report(test_data, name="Test Dataset")

### Data Splitting, Scaling & PCA

In [74]:
# Split the data into features (X) and target (y)
x = train_data.drop('Sales', axis=1)  # Drop the target column from the features
y = train_data['Sales']  # Extract the target column

# Split the data into training and testing sets (85% train, 15% test)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.15, random_state=42
)

In [75]:
# Apply StandardScaler to scale features for both training and testing sets
scaler = StandardScaler()

# Fit the scaler on the training features and transform them
x_train_scaled = scaler.fit_transform(x_train)

# Transform the testing features using the same scaler fitted on training data
x_test_scaled = scaler.transform(x_test)

In [78]:
# Apply PCA for dimensionality reduction after StandardScaler
pca = PCA(n_components=0.90)  # Keep 90% of variance

# Fit PCA on scaled training data and transform it
x_train_pca = pca.fit_transform(x_train_scaled)

# Transform the scaled testing data using the same PCA model
x_test_pca = pca.transform(x_test_scaled)

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [79]:
# Check the shape of the PCA-transformed training and testing data
print("Shape of X_train after PCA:", x_train_pca.shape)
print("Shape of X_test after PCA:", x_test_pca.shape)

NameError: name 'x_train_pca' is not defined

### Model Training & Evaluation

In [80]:
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model and evaluate it using the predefined function
train_evaluate_model(model, x_train, y_train, x_test, y_test)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [20]:
# Initialize the XGBoost Regressor model
model = XGBRegressor(random_state=42, use_label_encoder=False, eval_metric='rmse')

# Train and evaluate using the reusable function
train_evaluate_model(model, x_train, y_train, x_test, y_test)

XGBRegressor Performance Metrics

Training Set:
  Mean Absolute Error (MAE): 787.0561
  Root Mean Squared Error (RMSE): 1125.3441
  R-squared Score (R2): 0.9146

Test Set:
  Mean Absolute Error (MAE): 785.2866
  Root Mean Squared Error (RMSE): 1123.5722
  R-squared Score (R2): 0.9145


In [21]:
# Define parameter grid for Random Search
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.5],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0.5, 1, 1.5, 2],
}

# Randomized Search CV setup
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42, use_label_encoder=False, eval_metric='rmse'),
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the search on training data
random_search.fit(x_train, y_train)

# Get the best model from search
best_model = random_search.best_estimator_

# Train and evaluate using the provided function
train_evaluate_model(best_model, x_train, y_train, x_test, y_test)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
XGBRegressor Performance Metrics

Training Set:
  Mean Absolute Error (MAE): 359.6804
  Root Mean Squared Error (RMSE): 546.8981
  R-squared Score (R2): 0.9798

Test Set:
  Mean Absolute Error (MAE): 406.8700
  Root Mean Squared Error (RMSE): 639.5243
  R-squared Score (R2): 0.9723


### Save the trained model, PCA transformer, and Standard Scaler for future use

In [22]:
# Save the best trained regression model to disk
joblib.dump(best_model, 'XGB_Model.pkl')

# Save the fitted StandardScaler to disk
joblib.dump(scaler, 'standard_scaler.pkl')

# Save the fitted PCA transformer to disk
joblib.dump(pca, 'pca_transformer.pkl')

print("Best model, scaler, and PCA have been saved successfully.")

Best model, scaler, and PCA have been saved successfully.


### Save final test data with predictions

In [23]:
# Drop the 'Sales' column from the test_data DataFrame if it exists
test_data = test_data.drop(columns=['Sales'], axis=1)

In [26]:
test_data.shape

(41088, 27)

In [28]:
x_train.shape

(864627, 27)

In [29]:
# Make predictions
predictions = model.predict(test_data)

# Add predictions as a new column in test_data
test_data['Sales'] = predictions

# Show updated DataFrame (optional)
print(test_data.head())

   Store  Open  Promo  StateHoliday  SchoolHoliday  Assortment  compdistance  \
0      1   1.0      1             0              0           0        1270.0   
1      1   1.0      1             0              0           0        1270.0   
2      1   1.0      1             0              0           0        1270.0   
3      1   1.0      1             0              0           0        1270.0   
4      1   0.0      0             0              0           0        1270.0   

   compmonth  compyear  Promo2  ...  PromoInterval_Jan,Apr,Jul,Oct  \
0        9.0    2008.0       0  ...                              0   
1        9.0    2008.0       0  ...                              0   
2        9.0    2008.0       0  ...                              0   
3        9.0    2008.0       0  ...                              0   
4        9.0    2008.0       0  ...                              0   

   PromoInterval_Mar,Jun,Sept,Dec  PromoInterval_noPromo  Day_name_Monday  \
0                    

In [ ]:
# Export the test data along with the predicted sales to a CSV file for future reporting or analysis
test_data.to_csv('XGB_TestData_With_Predicted_Sales.csv', index=False)